In [6]:
import os
import csv

from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Lambda, Cropping2D
from tqdm import tqdm

samples = []
with open('./data/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None) 
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        #shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                #for i in range(0,3):
                    #camera= np.random.choice(['center', 'left', 'right'])
                    #if(camera=='center'):
                        name = './data/data/IMG/'+batch_sample[0].split('/')[-1]
                        #print(name)
                        center_image = cv2.imread(name)
                        center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                        #print(center_image.shape)
                        center_angle = float(batch_sample[3])
                        #print(center_angle)
                        images.append(center_image)
                        angles.append(center_angle)
                        images.append(cv2.flip(center_image,1))
                        angles.append(center_angle * -1)
                    
                    #if(camera=='left'):
                        
                        name = './data/data/IMG/'+batch_sample[1].split('/')[-1]
                        #print(name)
                        center_image = cv2.imread(name)
                        #print(center_image.shape)
                        #print(center_angle)
                        center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                        images.append(center_image)
                        center_angle = float(batch_sample[3])
                        angles.append(center_angle+0.2)
                        images.append(cv2.flip(center_image,1))
                        angles.append((center_angle + 0.2) * -1)
                    
                    #if(camera=='right'):
                        
                        name = './data/data/IMG/'+batch_sample[2].split('/')[-1]
                        #print(name)
                        center_image = cv2.imread(name)
                        center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                        #print(center_image.shape)
                        #print(center_angle)
                        images.append(center_image)
                        center_angle = float(batch_sample[3])

                        angles.append(center_angle-0.2)
                        
                        images.append(cv2.flip(center_image,1))
                        angles.append((center_angle - 0.2) * -1)
                    

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
#model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(3, 160, 320), output_shape=(3, 80, 320)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Convolution2D(24,5,5,subsample=(2,2)))
model.add(Activation('elu'))
model.add(Convolution2D(36,5,5,subsample=(2,2)))
model.add(Activation('elu'))
model.add(Convolution2D(48,5,5,subsample=(2,2)))
model.add(Activation('elu'))
model.add(Convolution2D(64,3,3))

model.add(Activation('elu'))
model.add(Convolution2D(64,3,3))

model.add(Activation('elu'))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('elu'))
model.add(Dense(50))
model.add(Activation('elu'))
model.add(Dense(10))
model.add(Activation('elu'))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
#model.fit(X_train,y_train,nb_epoch=5,validation_split=0.2,shuffle=True)

print(np.shape(train_samples))
#model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, validation_steps=len(validation_samples), nb_epoch=3,verbose=1)

model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator,   nb_val_samples=len(validation_samples), nb_epoch=8, verbose=2)
model.save('model.h5')
print('Done')

"""
If the above code throw exceptions, try 
model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
validation_data=validation_generator, validation_steps=len(validation_samples), epochs=5, verbose = 1)
"""



(6428, 7)
Epoch 1/8


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


12s - loss: 0.0334 - val_loss: 0.0314
Epoch 2/8
12s - loss: 0.0234 - val_loss: 0.0275
Epoch 3/8
12s - loss: 0.0247 - val_loss: 0.0205
Epoch 4/8
12s - loss: 0.0234 - val_loss: 0.0220
Epoch 5/8
12s - loss: 0.0196 - val_loss: 0.0201
Epoch 6/8
12s - loss: 0.0182 - val_loss: 0.0171
Epoch 7/8
12s - loss: 0.0170 - val_loss: 0.0186
Epoch 8/8
12s - loss: 0.0188 - val_loss: 0.0225
Done


'\nIf the above code throw exceptions, try \nmodel.fit_generator(train_generator, steps_per_epoch= len(train_samples),\nvalidation_data=validation_generator, validation_steps=len(validation_samples), epochs=5, verbose = 1)\n'